In [32]:
# test data
tdata = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""
tdata2 = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags."""

In [5]:
from autoloader import Autoloader
puz_data_url = "https://adventofcode.com/2020/day/7/input"
al = Autoloader(puz_data_url)
al.connect()
data = al.fetch()

In [35]:
class bag_rules():
    def __init__(self, data):
        self.bags = {}
        for line in data.splitlines():
            p, d = line.split("contain")
            if p[:-2] not in self.bags:
                self.bags[p[:-2]] = bag(p[:-2])
            
            self.bags[p[:-2]].add_rules(d, self.bags)
    def bag_count(self, t_bag):
        count = 0
        for b in self.bags:
            if self.bags[b].find_bag(t_bag):
                count += 1
        return count
    def full_bag_count(self, t_bag):
        return self.bags[t_bag].get_full_count() - 1
        
            
class bag():
    def __init__(self, name):
        self.name = name
        self.rules = {}

    def add_rules(self, rules_str, bags):
        for rule in rules_str.split(','):
            words = rule.split()
            if words[0] == 'no':
                continue
            num = int(words[0])
            t_bag = " ".join(words[1:]).replace('.', '')
            if num != 1:
                t_bag = t_bag[:-1]
            if t_bag not in bags:
                bags[t_bag] = bag(t_bag)
            self.rules[t_bag] = [num, bags[t_bag]]

    def find_bag(self, t_bag):
        if t_bag in self.rules:
            return True
        for r in self.rules:
            if self.rules[r][1].find_bag(t_bag):
                return True
        return False

    def get_full_count(self):
        count = 1
        for r in self.rules:
            count += (self.rules[r][0] * self.rules[r][1].get_full_count())
        return count


        
br = bag_rules(data)
br.bag_count("shiny gold bag")    
br.full_bag_count("shiny gold bag")

7867

In [26]:
class d7():
    def __init__(self, data):
        pass
    

Day 8

In [18]:
from autoloader import Autoloader
puz_data_url = "https://adventofcode.com/2020/day/8/input"
al = Autoloader(puz_data_url)
al.connect()
data = al.fetch()

In [6]:
data = """nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6"""

In [46]:
sdata = data.splitlines()


In [44]:
class HandheldDoneError(Exception):
    pass
class HandheldBoundError(Exception):
    pass
class handheld():
    def __init__(self, d):
        self.ii = 0
        self.d = []
        self.accumu = 0
        self.s_his = set()
        self.com_dict = {
            "nop" : self.nop,
            "acc" : self.acc,
            "jmp" : self.jmp
        }
        for l in d:
            c, v = l.split()
            self.d.append((self.com_dict[c], int(v)))
        self.step_size = 1
    def nop(self, _):
        pass
    def acc(self, val):
        self.accumu += val
    def jmp(self, val):
        self.step_size = val
    
    def step_hist(self):
        if self.ii in self.s_his:
            return False
        self.s_his.add(self.ii)
        self.step()
        return True
    
    def step(self):
        if self.ii == len(self.d):
            raise HandheldDoneError
        elif self.ii > len(self.d):
            raise HandheldBoundError
        self.d[self.ii][0](self.d[self.ii][1])
        self.ii += self.step_size
        self.step_size = 1


In [30]:
def is_inf(cur_data):
    try:
        hh = handheld(cur_data)

        first_trip = True

        while first_trip:
            first_trip = hh.step_hist()
        return True, 0
    except HandheldDoneError:
        return False, hh.accumu
    except HandheldBoundError:
        return True, 0
    


In [33]:
t_data = """nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6"""


In [34]:
sdata = t_data.splitlines()

In [48]:
def try_pos(sd0, ii):
    sd = sd0[:]
    t = sd[ii].split()[0]
    if t == 'acc':
        return None
    if t == 'nop':
        sd[ii] = sd[ii].replace('nop', 'jmp')
    elif t == 'jmp':
        sd[ii] = sd[ii].replace('jmp', 'nop')
    ans = is_inf(sd)
    if ans[0]:
        return None
    else:
        return ans[1]

    

In [49]:
res = None
ii = 0
while res == None:
    res = try_pos(sdata, ii)
    ii += 1
print(res)

1056


In [59]:
from autoloader import Autoloader
puz_data_url = "https://adventofcode.com/2020/day/9/input"
al = Autoloader(puz_data_url)
al.connect()
data = al.fetch()

In [50]:
tdata = """35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576"""

In [1]:
from collections import deque
preamble = 5
class xmas():
    def __init__(self, preamble_l, data):
        sdata = data.splitlines()
        self.cur_vals = deque((int(x) for x in sdata[:preamble_l]), preamble_l)
        self.rdata = deque((int(x) for x in sdata[preamble_l:]))
        self.full_data = [int(x) for x in sdata]
    def step(self):
        cur_number = self.rdata.popleft()
        valid = False
        cur_val_list = list(self.cur_vals)
        for ii, val1 in enumerate(cur_val_list):
            for val2  in cur_val_list[ii:]:
                if val1+val2 == cur_number and val1 != val2:
                    valid = True
        self.cur_vals.popleft()
        self.cur_vals.append(cur_number)
        return valid, cur_number
    def find_weakness(self, t_num):
        cur_sum = 0
        s_index = 0
        for ii, val in enumerate(self.full_data):
            cur_sum += val
            while cur_sum > t_num:
                cur_sum -= self.full_data[s_index]
                s_index += 1
            if cur_sum == t_num:
                return max(self.full_data[s_index:ii+1]) + min(self.full_data[s_index:ii+1])
            
                
x = xmas(25, data)
valid = True
while(valid == True):
    valid, num = x.step()
print(x.find_weakness(num))



NameError: name 'data' is not defined

In [14]:
tdata2 = """28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3"""

tdata1 = """16
10
15
5
1
11
7
19
6
12
4"""

In [11]:
from autoloader import Autoloader
puz_data_url = "https://adventofcode.com/2020/day/10/input"
al = Autoloader(puz_data_url)
al.connect()
data = al.fetch()

In [42]:
import numpy as np
def count_steps_subseq(subseq):
    if len(subseq) <= 2:
        return 1
    dists = get_dists(subseq)
    paths = np.zeros(subseq.shape, dtype=np.uint32)
    paths[0] = 1
    for ii in range(subseq.shape[0]):
        for kk in range(ii-dists[ii], ii):
            paths[ii] += paths[kk]
    return paths[-1]

def get_dists(subseq):
    dists = np.zeros(subseq.shape, dtype=np.uint32)
    for ii in range(subseq.shape[0]):
        for kk in range(ii-3, ii):
            if kk < 0:
                continue
            if subseq[ii] - subseq[kk] <= 3:
                dists[ii] += 1
    return dists
                


class inport_day10():
    def __init__(self, i_data):
        adapt = [0]
        for line in i_data.splitlines():
            adapt.append(int(line))

        self.adapt_sorted = np.array(sorted(adapt))
    def count_steps(self):
        vol = 0
        difs = [0,0,0,0]
        for step in self.adapt_sorted:
            cur_dif = step - vol
            difs[cur_dif] += 1
            vol = step
        self.dif_arr = difs
        return difs
    def count_poss_steps(self):
        subseq_gen = self.get_subseq()
        cur_count = 1
        try:
            while True:
                indexes = next(subseq_gen)
                cur_slice = self.adapt_sorted[indexes[0]:indexes[1]]
                cur_count *= count_steps_subseq(cur_slice)
        except StopIteration:
            return cur_count

    def get_subseq(self):
        vol = 0
        ii = 0
        for kk, step in enumerate(self.adapt_sorted):
            if step - vol == 3:
                yield (ii,kk)
                ii = kk
            vol = step


        
sd = inport_day10(data)
# _, ones, _, threes = sd.count_steps()
# threes += 1
print(sd.count_poss_steps())
#print(ones, threes)
#print(f'{ones*threes}')


14173478093824


In [45]:
tdata = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL"""

In [104]:
from autoloader import Autoloader
puz_data_url = "https://adventofcode.com/2020/day/11/input"
al = Autoloader(puz_data_url)
al.connect()
data = al.fetch()

In [112]:
import numpy as np
import itertools
import functools
class seatLayout():
    def __init__(self, i_pos):
        i_pos_list = i_pos.splitlines()
        self.board = np.zeros((len(i_pos_list), len(i_pos_list[0])), dtype=np.int32)
        for r, line in enumerate(i_pos.splitlines()):
            for c, seat in enumerate(line):
                if seat == 'L':
                    self.board[r,c] = 0
                elif seat == '.':
                    self.board[r,c] = -1
                else:
                    self.board[r,c] = 1
        self.build_neighbor_arr()

    def count_neighbors(self, pos):
        dirs = set(itertools.permutations([0,1,1,-1,-1], 2))
        count = 0
        for dir in dirs:
            if self.check_occ(pos, dir):
                count += 1
        return count

    @functools.cache
    def check_occ(self, pos, dir):
        t_pos = pos + dir
        state = 0
        if t_pos[0] < 0 or t_pos[1] < 0 :
            return False
        try:
            state = self.board[tuple(t_pos)]
            if state >= 1:
                return True
        except IndexError:
            return False
        return False
    def build_neighbor_arr(self):
        n_arr = self.board.copy()
        for ii in range(self.board.shape[0]):
            for kk in range(self.board.shape[1]):
                if self.board[ii,kk] < 0:
                    continue
                n_arr[ii,kk] = self.count_neighbors(np.array((ii,kk), dtype=np.int32))
        self.n_arr = n_arr
    
    def rule1(self, pos):
        if self.board[tuple(pos)] != 0:
            return False
        if self.n_arr[tuple(pos)] != 0:
            return False
        return True
    def rule2(self, pos):
        if self.board[tuple(pos)] <= 0:
            return False
        if self.n_arr[tuple(pos)] < 4:
            return False
        return True
    def check_rules(self):
        next_board = self.board.copy()
        for ii in range(self.board.shape[0]):
            for kk in range(self.board.shape[1]):
                if self.board[ii,kk] < 0:
                    continue
                if self.rule1((ii,kk)):
                    next_board[ii,kk] = 1
                elif self.rule2((ii,kk)):
                    next_board[ii,kk] = 0
        if np.array_equal(self.board, next_board):
            return False
        self.board = next_board
        return True
    def find_stab(self):
        unstab = True
        while(unstab):
            unstab = self.check_rules()
            self.build_neighbor_arr()

    def __abs__(self):
        count = 0
        for ii in range(self.board.shape[0]):
            for kk in range(self.board.shape[1]):
                if self.board[ii,kk] < 0:
                    continue
                if self.board[ii,kk] > 0:
                    count += 1
        return count
    def __str__(self):
        print("----------")
        for ii in range(self.board.shape[0]):
            for kk in range(self.board.shape[1]):
                if self.board[ii,kk] < 0:
                    print(" ", end="")
                elif self.board[ii,kk] == 0:
                    print("o", end="")
                elif self.board[ii,kk] > 0:
                    print("X", end="")
            print()
        print("----------")
        return ""
    
class visable_seat_Layout(seatLayout):
    def rule2(self, pos):
        if self.board[tuple(pos)] <= 0:
            return False
        if self.n_arr[tuple(pos)] < 5:
            return False
        return True
    def check_occ(self, pos, dir):
        t_pos = pos+dir
        state = 0
        if t_pos[0] < 0 or t_pos[1] < 0 :
            return False
        try:
            state = self.board[tuple(t_pos)]
            if state == 0:
                return False
            elif state >= 1:
                return True
            elif state < 0:
                return self.check_occ(t_pos, dir)
        except IndexError:
            return False
        return False





                    

            


pos = np.array((0,9))
                

s = visable_seat_Layout(data)
s.find_stab()
print(abs(s))


AttributeError: module 'functools' has no attribute 'cache'